In [1]:
from astropy.table import Table
from matplotlib import pyplot as plt
%matplotlib inline
import os
import numpy as np
from astropy.io.ascii import masked
from astropy.coordinates import Angle
from astropy.io import ascii
import glob
from astropy.io import fits
import wget
import matplotlib.image as mpimg
from astropy.wcs import WCS
from scipy.stats import scoreatpercentile
from astropy.visualization import simple_norm
from reproject import reproject_interp
import sys
from IPython.display import clear_output
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from photutils.aperture import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization import ImageNormalize
from astropy.visualization import LogStretch
from astropy.wcs import WCS
import astropy.units as u
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import Angle
from scipy import stats
from astropy.visualization import MinMaxInterval
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib import pyplot as plt
from matplotlib import colors
from astropy.stats import gaussian_sigma_to_fwhm
import pandas as pd
from reproject import reproject_interp
from astropy.nddata import CCDData
import warnings
warnings.filterwarnings('ignore')

mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']

from photutils.isophote import EllipseGeometry
from photutils.aperture import EllipticalAperture

#define an empty dictionary that will contain the EllipseGeometry instance
geometry = {}
initparams = {}
#initialize dictionary for half-light radii
rhalfpix = {}
rhalfasec = {}
from photutils.isophote import Ellipse
from photutils import aperture_photometry
#initialize dictionary for ellipse fitting
ellipse = {}
isolist = {}
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
mycolors = plt.rcParams['axes.prop_cycle'].by_key()['color']
#Set the home path

os.environ['HOME'] ='C:/Users/USER/Documents/GitHub'
homedir = os.getenv("HOME")
tabledir = homedir+'/Virgo/tables/'
plotdir = homedir+'/Virgo/plots/'
datadir = homedir+'/masking/'

In [2]:
def imdisplay(image,x,y, sma, ellip,pa,width=100, height=100, v1perc=10, v2perc=95, logscale=True):
    '''
    display an image 
    OPTIONAL KEYWORD PARAMETERS
    v1perc: one end of the colormap assigned to the v1perc percent lowest flux 
    v2perc: the other end of the colormap assigned to the v2perc percent highest flux    
    '''
    # make sure image is an np array
    nimage = np.array(image)
    # determine the pixel values at the 10th and 95th percentile
    v1 = scoreatpercentile(nimage,v1perc)
    v2 = scoreatpercentile(nimage,v2perc)
    # display using imshow
    #
    # you can play with alternate cmaps in the function below, such as "viridis" or "gray"
    # The 'gray_r' color map reverses the color-scale so that dark display pixels are the brightest in the image
    #
    # vmin and vmax set the min and max pixel values that
    # will be mapped to the extremes of the colormap
    print(v1,v2)
    norm = None
    if (logscale):
        norm = ImageNormalize(vmin=v1, vmax=v2, stretch=LogStretch())
    else:
        norm = ImageNormalize(vmin=v1, vmax=v2)
    im = ax.imshow(image, origin='lower', norm=norm)
    aper = EllipticalAperture((x, y), sma,
                              (ellip)*sma, pa)
    # Adjust the view to focus on (x, y) without cropping the image
    x_min = max(x - width // 2, 0)
    x_max = min(x + width // 2, nimage.shape[1])
    y_min = max(y - height // 2, 0)
    y_max = min(y + height // 2, nimage.shape[0])
    # Set the axis limits so that (x, y) is at the center
    ax.set_xlim(x - width // 2, x + width // 2)
    ax.set_ylim(y - height // 2, y + height // 2)
    # Hide axis ticks and labels
    plt.axis('off')    
    aper.plot(color='red')
    return fig,ax
    #fig.colorbar(fraction=.08)
def find_files(destination_folder, partial_name):
    matching_files = []

    for root, dirs, files in os.walk(destination_folder):
        for file in files:
            if partial_name.lower() in file.lower():
                matching_files.append(os.path.join(root, file))

    return matching_files 
# Function to overlay mask on original FITS image and save as a new FITS file
def overlay_mask_on_fits(fits_file, mask_file, csv_file, output_fits, n, coords_x, coords_y):
    # Load the original FITS image and its WCS
    original_fits_data, original_fits_header = fits.getdata(fits_file, header=True)
    original_wcs = WCS(original_fits_header)
    # Load the mask FITS image and its WCS
    mask_fits_data, mask_fits_header = fits.getdata(mask_file, header=True)
    mask_wcs = WCS(mask_fits_header)
    # Extract the central X and Y pixel coordinates (assuming columns are named 'x' and 'y')
    central_x = int(coords_x)
    central_y = int(coords_y) 
    # Reproject the mask to the original FITS image WCS
    reprojected_mask, footprint = reproject_interp((mask_fits_data, mask_wcs), original_wcs, shape_out=original_fits_data.shape)    
    # Ensure that the central pixel coordinates are within the bounds of the original image
    height, width = original_fits_data.shape
    central_x = np.clip(central_x, 0, width - 1)
    central_y = np.clip(central_y, 0, height - 1)
    # Create a copy of the original FITS data and overlay the reprojected mask
    combined_data = original_fits_data.copy()
    # Overlay the mask where the mask is non-zero (assumes mask values are 0 or 1)
    combined_data[reprojected_mask > 0] = np.max(original_fits_data)  # Overlay mask pixels
    # Ensure that the central pixel is not masked unless explicitly masked in the mask file
    if reprojected_mask[central_y, central_x] == 0:
        # If the mask does not cover the central pixel, restore the original value
        combined_data[central_y, central_x] = original_fits_data[central_y, central_x]
    #else:
        # Optionally, you can mark the central pixel differently if the mask explicitly masks it
        #print(f"Central pixel ({central_x}, {central_y}) is masked in the mask file.")
    # Save the result as a new FITS file
    fits.writeto(output_fits, combined_data, original_fits_header, overwrite=True)

In [15]:
#print out central pixels
# Input CSV file with galaxy data
csv_file = tabledir + '/Photometrytesting.csv'
galaxy_data = pd.read_csv(csv_file)

# Prepare output CSV
output_data = []

# Process each galaxy
for i in range(len(galaxy_data)):
    galaxy_name = str(galaxy_data['GALAXY'][i])
    RA = galaxy_data['RA_MOMENT'][i]
    DEC = galaxy_data['DEC_MOMENT'][i]
    VFID = f"VFID{int(galaxy_data['VF_ID'][i]):04d}"

    # Paths for the images
    base_path = os.path.join(datadir, 'pipeline', galaxy_name)
    mask_path = os.path.join(datadir, 'masks')
    
    # Search for the HPPUNIMAPB FITS file dynamically
    destination_folder = os.path.join(datadir, 'pipeline', galaxy_name, 'HPPUNIMAPB')
    partial_name = 'hpacs_25HPPUNIMAPB'
    found_files = find_files(destination_folder, partial_name)
    hppunimapb_image_path = found_files[0] if found_files else None

    # Search for the HPPUNIMAPG FITS file dynamically
    destination_folder = os.path.join(datadir, 'pipeline', galaxy_name, 'HPPUNIMAPG')
    partial_name = 'hpacs_25HPPUNIMAPB'
    found_files = find_files(destination_folder, partial_name)
    hppunimapg_image_path = found_files[0] if found_files else None
    # Search for the HPPUNIMAPR FITS file dynamically
    destination_folder = os.path.join(datadir, 'pipeline', galaxy_name, 'HPPUNIMAPR')
    partial_name = 'hpacs_25HPPUNIMAPR'
    found_files = find_files(destination_folder, partial_name)
    hppunimapr_image_path = found_files[0] if found_files else None

    # Check and process WISE mask image
    wise_mask_image_path = os.path.join(mask_path, f'{galaxy_name}-custom-image-wise-mask.fits')
    wise_mask_x, wise_mask_y = None, None
    if os.path.exists(wise_mask_image_path):
        wise_mask_data, wise_mask_header = fits.getdata(wise_mask_image_path, header=True)
        wise_mask_wcs = WCS(wise_mask_header)
        wise_mask_x, wise_mask_y = wise_mask_wcs.all_world2pix(RA, DEC, 0)
    else:
        # If the original WISE mask is not found, check for the alternative '-custom-image-r-mask.fits'
        wise_mask_image_path_r = os.path.join(mask_path, f'{galaxy_name}-custom-image-r-mask.fits')
    
        if os.path.exists(wise_mask_image_path_r):
            wise_mask_data, wise_mask_header = fits.getdata(wise_mask_image_path_r, header=True)
            wise_mask_wcs = WCS(wise_mask_header)
            wise_mask_x, wise_mask_y = wise_mask_wcs.all_world2pix(RA, DEC, 0)
        else:
            # If neither WISE mask is found, set the pixel coordinates to None
            wise_mask_x, wise_mask_y = None, None

    # Initialize pixel coordinates
    hppunimapb_x, hppunimapb_y = None, None
    hppunimapg_x, hppunimapg_y = None, None
    hppunimapr_x, hppunimapr_y = None, None

    # Check and process HPPUNIMAPB image
    if hppunimapb_image_path and os.path.exists(hppunimapb_image_path):
        hppunimapb_data, hppunimapb_header = fits.getdata(hppunimapb_image_path, header=True)
        hppunimapb_wcs = WCS(hppunimapb_header)
        hppunimapb_x, hppunimapb_y = hppunimapb_wcs.all_world2pix(RA, DEC, 0)
    # Check and process HPPUNIMAPB image
    if hppunimapg_image_path and os.path.exists(hppunimapg_image_path):
        hppunimapg_data, hppunimapg_header = fits.getdata(hppunimapg_image_path, header=True)
        hppunimapg_wcs = WCS(hppunimapg_header)
        hppunimapg_x, hppunimapg_y = hppunimapg_wcs.all_world2pix(RA, DEC, 0)
    # Check and process HPPUNIMAPB image
    if hppunimapr_image_path and os.path.exists(hppunimapr_image_path):
        hppunimapr_data, hppunimapr_header = fits.getdata(hppunimapr_image_path, header=True)
        hppunimapr_wcs = WCS(hppunimapr_header)
        hppunimapr_x, hppunimapr_y = hppunimapr_wcs.all_world2pix(RA, DEC, 0)

    # Append the results to the output data
    output_data.append({
        'VFID': VFID,
        'Galaxy': galaxy_name,
        'RA': RA,
        'DEC': DEC,
        'HPPUNIMAPB_X': hppunimapb_x,
        'HPPUNIMAPB_Y': hppunimapb_y,
        'HPPUNIMAPG_X': hppunimapg_x,
        'HPPUNIMAPG_Y': hppunimapg_y,
        'HPPUNIMAPR_X': hppunimapr_x,
        'HPPUNIMAPR_Y': hppunimapr_y,
        'WISE_Mask_X': wise_mask_x,
        'WISE_Mask_Y': wise_mask_y
    })

# Convert to DataFrame and save to CSV
output_df = pd.DataFrame(output_data)
output_csv_path = datadir + '/pixel_coordinates.csv'
output_df.to_csv(output_csv_path, index=False)
print('done')

done


In [16]:
#imposing the masks onto the images
import os
import pandas as pd
from astropy.io import fits
from astropy.table import Table

# File paths
csv_file = os.path.join(datadir, 'pixel_coordinates.csv')
galaxy = Table.read(csv_file)

for i in range(len(galaxy)):
    galaxy_name = str(galaxy['Galaxy'][i])
    path = os.path.join(datadir, 'pipeline', galaxy_name)
    VFID = str(galaxy['VFID'][i])

    if os.path.exists(path):
        for color in ['B', 'G', 'R']:  # Loop over the three color channels
            destination_folder = os.path.join(path, f'HPPUNIMAP{color}')
            if color in ['B', 'G']:  # Blue and Green bands
                partial_name = f'hpacs_25HPPUNIMAPB'
            else:  # Red band
                partial_name = f'hpacs_25HPPUNIMAPR'
            found_files = find_files(destination_folder, partial_name)

            if found_files:
                found_file = found_files[0]

                # Mask file paths
                wise_mask_file = os.path.join(datadir, 'masks', f'{galaxy_name}-custom-image-wise-mask.fits')
                r_mask_file = os.path.join(datadir, 'masks', f'{galaxy_name}-custom-image-r-mask.fits')

                # Output FITS file path
                output_fits = os.path.join(datadir, 'masked', f'{galaxy_name}_masked{color}.fits')

                # Read the CSV file for the central pixel coordinates
                coords_data = pd.read_csv(csv_file)
                coords_x = coords_data[f'HPPUNIMAP{color}_X'][i]
                coords_y = coords_data[f'HPPUNIMAP{color}_Y'][i]

                # Check if the wise-mask file exists, otherwise check for the r-mask
                if os.path.exists(wise_mask_file):
                    # If wise-mask exists, overlay it
                    overlay_mask_on_fits(found_file, wise_mask_file, csv_file, output_fits, i, coords_x, coords_y)
                elif os.path.exists(r_mask_file):
                    # If r-mask exists, overlay it
                    overlay_mask_on_fits(found_file, r_mask_file, csv_file, output_fits, i, coords_x, coords_y)
                else:
                    # If no mask files exist, simply copy the original FITS file to the output location
                    if not os.path.exists(os.path.dirname(output_fits)):
                        os.makedirs(os.path.dirname(output_fits))
                    with fits.open(found_file) as hdul:
                        hdul.writeto(output_fits, overwrite=True)
                    print(f"Mask file not found for {galaxy_name} or {VFID}. Saved original FITS as {output_fits}")


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:/Users/USER/Documents/GitHub/masking/masked\\NGC5713_maskedB.fits'

In [17]:
#doing photometry with these masked galaxies
# Load the CSV containing v1 and v2 values for each color

# Ensure the table has columns 'v1_blue', 'v2_blue', 'v1_green', 'v2_green', 'v1_red', 'v2_red'
csv_file = tabledir + '/Photometrytesting.csv'
csv_file2 = os.path.join(datadir, 'pixel_coordinates.csv')
galaxy = Table.read(csv_file)

# Add flux and masked percentage columns
galaxy['70Flux_AP06'] = np.zeros(len(galaxy))
galaxy['100Flux_AP06'] = np.zeros(len(galaxy))
galaxy['160Flux_AP06'] = np.zeros(len(galaxy))
galaxy['good_perc_blue_green'] = np.zeros(len(galaxy))  # Column for blue/green percentage
galaxy['good_perc_red'] = np.zeros(len(galaxy))  # Column for red percentage

# Determine the length of the galaxy table
galaxy_length = len(galaxy)

# Arrays for flux and masked percentage
ap06fluxblue = np.zeros(galaxy_length, dtype='f')
ap06fluxgreen = np.zeros(galaxy_length, dtype='f')
ap06fluxred = np.zeros(galaxy_length, dtype='f')
good_perc_blue_green = np.zeros(galaxy_length, dtype='f')
good_perc_red = np.zeros(galaxy_length, dtype='f')

for i in range(galaxy_length):
    galaxy_name = str(galaxy['GALAXY'][i])
    VFID = f"VFID{int(galaxy['VF_ID'][i]):04d}"
    RA = galaxy['RA_MOMENT'][i]
    DEC = galaxy['DEC_MOMENT'][i]
    EPLI = galaxy['BA_MOMENT'][i]
    PAN = (galaxy['PA_MOMENT'][i] + 90) * np.pi / 180
    SMAO = galaxy['SMA_AP06'][i]

    # Process the FITS files for blue, green, and red fluxes
    for color, flux_col, flux_array, fits_suffix, is_red in zip(
            ['blue', 'green', 'red'],
            ['70Flux_AP06', '100Flux_AP06', '160Flux_AP06'],
            [ap06fluxblue, ap06fluxgreen, ap06fluxred],
            ['maskedB.fits', 'maskedG.fits', 'maskedR.fits'],
            [False, False, True]):  # `True` for red, `False` for blue/green
        
        # Get pre-calculated v1 and v2 for the current color from the v1_v2_table
        #v1 = v1_v2_table[v1_col][i]
        
        # Construct the output FITS file path
        output_fits = os.path.join(datadir, 'masked', f'{galaxy_name}_{fits_suffix}')
        
        # Check if the FITS file exists before proceeding
        if os.path.exists(output_fits):
            placeholder = fits.open(output_fits)
            data, header = fits.getdata(output_fits, header=True)
            pixscale = 1.60000001784 if not is_red else 3.20000003568  # Adjust pixscale for red
            SMAN = (SMAO * 0.262) / np.abs(pixscale)
            wcs = WCS(header)
            x0, y0 = wcs.all_world2pix(RA, DEC, 0)
            initparams[galaxy_name] = {
                'xcen': x0,
                'ycen': y0,
                'sma': SMAN,
                'ellip': EPLI,
                'PA': PAN
            }
            fig, ax = plt.subplots(figsize=(10, 10))
            nimage = np.array(data)
            for v1perc in np.arange(0.1, 40.0, 0.1):
                v1 = scoreatpercentile(nimage, v1perc)
                if v1 > 0:
                    break
            for v2perc in np.arange(99.9, 20.0, -0.1):
                v2 = scoreatpercentile(nimage, v2perc)
                if not np.isnan(v2):
                    break                
            # Call imdisplay with pre-calculated v1 and v2 for the current color
            if is_red:
                imdisplay(data, x0, y0, SMAN, EPLI, PAN,200,200, v1perc, v2perc, logscale=True)
            else:
                imdisplay(data, x0, y0, SMAN, EPLI, PAN,350,350, v1perc, v2perc, logscale=True)
            
            # Create the elliptical aperture based on galaxy parameters
            aper = EllipticalAperture((x0, y0), SMAN, SMAN * EPLI, PAN)
            
            # Mask calculation based on NaN values (masked pixels are NaN)
            aperture_mask = aper.to_mask(method='center')
            aperture_data = aperture_mask.multiply(nimage)  # Extract the region within the aperture
            
            # Count total pixels within the aperture
            total_pixels = np.isfinite(aperture_data).sum()  # Total pixels that are not NaN
            empty_pixels = np.isnan(aperture_data).sum()  # Already masked pixels (NaN values)
            
            # Calculate the percentage of masked (NaN) pixels
            if total_pixels + empty_pixels > 0:
                good_percentage = 1 - (empty_pixels / (total_pixels + empty_pixels))
            else:
                good_percentage = 1  # If no pixels are counted, default to 0 
            # Assign the good percentage based on the band
            if is_red:
                good_perc_red[i] = good_percentage
                galaxy['good_perc_red'][i] = good_percentage
            else:
                good_perc_blue_green[i] = good_percentage
                galaxy['good_perc_blue_green'][i] = good_percentage
            # Replace NaN values with 0 in the whole image before photometry
            nimage_nonan = np.nan_to_num(nimage, nan=0)

            # Create the elliptical aperture based on galaxy parameters
            aper = EllipticalAperture((x0, y0), SMAN, SMAN * EPLI, PAN)

            # Perform photometry on the original image with NaNs treated as 0
            phot_table = aperture_photometry(nimage_nonan, aper)

            # Store the flux result
            flux_array[i] = phot_table['aperture_sum'][0]
            galaxy[flux_col][i] = flux_array[i]
            plt.subplots_adjust(wspace=0, hspace=0)
            plt.savefig(datadir + '/pngs/' + VFID + '-' + galaxy_name + color+'.png', dpi=150)
            plt.close()

# Save the updated table with photometry fluxes and masked percentages
galaxy.write(tabledir + '/Photometrytesting2.csv', format='csv', overwrite=True)
print('done')

5.643724070978631e-07 0.0031125775687808237
2.21401137423522e-07 0.00657667959547661
3.789014415815524e-06 0.006997645934175211
2.474536304362248e-07 0.004583491589427613
6.520480383188382e-07 0.007384169423892876
2.3342040367424653e-06 0.011438516967915428
1.205237029353192e-06 0.006128234870750446
1.123524474678738e-06 0.007684906676033609
1.5495519619435207e-06 0.014349428075646833
6.910762749614291e-08 0.005798332206911989
1.0658379760570876e-06 0.01516897153855387
5.770387360826307e-07 0.031749210157496474
2.3823520750738902e-06 0.0054783142791238245
6.930370582267839e-07 0.0028871235531069033
4.23974692239428e-06 0.014217163146244646
7.618330710112844e-08 0.004207936950058951
1.5707383863634146e-07 0.006318833543049902
1.1551637726370595e-06 0.0035005807877622783
1.7387413536199463e-06 0.018149568423085585
7.590325549246271e-07 0.04343015213371669
3.0405470170092345e-07 0.06536444458497842
1.7983188736253568e-06 0.5004834803142005
8.548668120056937e-07 1.0595573135087786
2.871500